In [1]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ry2a_5t6/unsloth_ea7c198841ba49e19dcf31b1808111fb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ry2a_5t6/unsloth_ea7c198841ba49e19dcf31b1808111fb
  Resolved https://github.com/unslothai/unsloth.git to commit c7fc68c5e0a296ed7d3e16449e055ded20713bad
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from unsloth import FastLanguageModel

In [6]:
import torch
import pandas as pd
import torch.nn.functional as F

In [7]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [8]:
import numpy as np
from tqdm.auto import tqdm

In [9]:
import re
import string
import torch
import pandas as pd
from functools import lru_cache
from collections import defaultdict

In [10]:
def install_llama():
  from unsloth import FastLanguageModel
  max_seq_length = 2048
  dtype = None
  load_in_4bit = True


  fourbit_models = [
      "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
      "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
      "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
      "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
      "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
      "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
      "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
      "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
      "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
      "unsloth/Phi-3-medium-4k-instruct",
      "unsloth/gemma-2-9b-bnb-4bit",
      "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
      ]

  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "unsloth/Meta-Llama-3.1-8B",
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = True
      )
  return model, tokenizer

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
model, tokenizer = install_llama()

==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [13]:
cyrillic_re = re.compile(r'[\u0400-\u04FF]')

def is_cyrillic(text):
    return bool(re.search(r'[а-яёА-ЯЁ]', text))

In [14]:
SEPARATORS = set([' ', ',', '.', ';', ':', '!', '?', '\n', '\t'])

In [15]:
def is_valid_token(token_str):
    token_str = token_str.strip()
    if is_cyrillic(token_str):
        return True
    if token_str in ["", " ", "\n", "\t"]:
        return True
    if re.match(r'^[\.,!?-]+$', token_str):
        return True
    return False

In [16]:
def is_space_token(token_str):
    """Проверяет, является ли токен пробелом"""
    return token_str.strip() == "" and token_str != ""


In [17]:
def ends_with_separator(token_str):
    """Проверяет, заканчивается ли токен на разделитель или пунктуацию"""
    return bool(re.search(r'[ .,!?;:"\']$', token_str))


In [18]:
def extract_separator(token_str):
  """
  Функция извлекает сепаратор из токена.
  """
  if token_str and token_str[0] in SEPARATORS:
      return token_str[0]
  if token_str and token_str[-1] in SEPARATORS:
      return token_str[-1]
  return None

In [19]:
def contains_separator(token_str):
    return (token_str and (token_str[0] in SEPARATORS or token_str[-1] in SEPARATORS))


In [20]:
def is_single_cyrillic_word(word):
    if not word or not word.strip():
        return False
    word = word.strip()
    return bool(re.match(r'^[а-яёА-ЯЁ-]+$', word))

In [21]:
def initialize_token_structures(tokenizer):
    separator_strings = [" ", ",", ".", ";", ":", "!", "?", "\n", "\t"]
    separator_ids = set()

    for sep in separator_strings:
        tokens = tokenizer.encode(sep, add_special_tokens=False)
        separator_ids.update(tokens)

    special_tokens = []
    if tokenizer.eos_token_id is not None:
        special_tokens.append(tokenizer.eos_token_id)
    if tokenizer.bos_token_id is not None:
        special_tokens.append(tokenizer.bos_token_id)
    if tokenizer.pad_token_id is not None:
        special_tokens.append(tokenizer.pad_token_id)
    if tokenizer.unk_token_id is not None:
        special_tokens.append(tokenizer.unk_token_id)

    separator_ids.update(special_tokens)

    original_vocab_size = tokenizer.vocab_size
    cyrillic_token_ids = set()

    for token_id in range(original_vocab_size):
        try:
            token_str = tokenizer.decode([token_id], skip_special_tokens=False)
            if cyrillic_re.search(token_str.strip()) or token_id in separator_ids:
                cyrillic_token_ids.add(token_id)
        except Exception:
            pass

    cyrillic_token_ids.update(special_tokens)
    cyrillic_token_ids = sorted(cyrillic_token_ids)

    new_id_to_old_id = {new_id: old_id for new_id, old_id in enumerate(cyrillic_token_ids)}
    old_id_to_new_id = {old_id: new_id for new_id, old_id in enumerate(cyrillic_token_ids)}

    print(f"Reduced vocabulary size: {len(cyrillic_token_ids)} / {original_vocab_size}")

    token_cache = {}
    for token_id in cyrillic_token_ids:
        token_str = tokenizer.decode([token_id])
        token_cache[token_id] = {
            'str': token_str,
            'is_cyrillic': is_cyrillic(token_str.strip()),
            'is_valid': is_valid_token(token_str.strip()),
            'is_separator': token_str.strip() in separator_strings or token_str.strip() in string.punctuation,
            'is_space': is_space_token(token_str),
            'ends_with_separator': ends_with_separator(token_str),
            'contains_separator': contains_separator(token_str)
        }

    return cyrillic_token_ids, new_id_to_old_id, old_id_to_new_id, token_cache

In [22]:
@lru_cache(maxsize=2000)
def get_next_token_probs(input_ids_tuple, cyrillic_token_ids_tuple, model, device):
    input_ids = torch.tensor(input_ids_tuple, device=device).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, return_dict=True)
        logits = outputs.logits[:, -1, :]
        reduced_logits = logits[:, list(cyrillic_token_ids_tuple)]
        probs = torch.softmax(reduced_logits, dim=-1).squeeze()
        return probs.cpu().numpy()

In [23]:
"начинаться с разделителя и тогда надо пролжолюаьь генерацию. Это сепаратор нужно убрать из начала слова и никуда не сохранять "
"если это не первый токен последовательности и он начинается с сепаратора, то нужно сохранить только сепартор и прекратить генерацию. Вероятность сепаратора брать, как вероятность всего токена "
"если токен это только сепаратор, то тогда нужно прекратить генерацию и сохранить вероятность этого токена "
"если токен содержит сепартор в конце, то буквенную часть нужно добавить к предыдущим токенам, а сепартор сохранить отдельно. Вероятность сепаратора считать как вероятность всего токена"

'если токен содержит сепартор в конце, то буквенную часть нужно добавить к предыдущим токенам, а сепартор сохранить отдельно. Вероятность сепаратора считать как вероятность всего токена'

In [24]:
def beam_search_completions3(sentence, tokenizer, model,
                          max_depth=10, beam_width=100, min_prob=1e-6,
                          cyrillic_token_ids=None, new_id_to_old_id=None,
                          token_cache=None, device=None):
    """
    Beam search с учётом сепараторов и мультитокеновых слов.
    Условия:
    1) Если первый токен начинается с сепаратора — убираем сепаратор и продолжаем генерацию.
    2) Если не первый токен и начинается с сепаратора — сохраняем текущее слово и прекращаем генерацию.
    3) Если токен — только сепаратор — прекращаем генерацию.
    4) Если сепаратор в конце токена — буквенную часть добавляем к слову и прекращаем генерацию.
    """
    if "<mask>" not in sentence:
        return None

    prefix, suffix = sentence.split("<mask>", 1)
    prompt = prefix.strip()

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    input_ids = inputs["input_ids"]

    cyrillic_token_ids_tuple = tuple(cyrillic_token_ids)

    completed_words = {}
    word_details = {}

    # Лучи: (input_ids, current_word_tokens, current_word_probs, sequence_prob, depth)
    beams = [(input_ids, [], [], 1.0, 0)]

    for depth in range(max_depth):
        if not beams:
            break

        new_beams = []

        for beam in beams:
            current_ids, current_word, current_word_probs, current_prob, current_depth = beam

            input_tuple = tuple(current_ids.squeeze().tolist())
            next_probs = get_next_token_probs(input_tuple, cyrillic_token_ids_tuple, model, device)

            top_indices = np.argsort(next_probs)[-beam_width:][::-1]

            for reduced_id in top_indices:
                prob = next_probs[reduced_id]
                total_prob = current_prob * prob

                if total_prob < min_prob:
                    continue

                token_id = new_id_to_old_id[reduced_id]
                token_info = token_cache[token_id]

                if not token_info['is_valid']:
                    continue

                token_str = token_info['str']
                new_ids = torch.cat([current_ids, torch.tensor([[token_id]], device=device)], dim=1)

                # Определяем сепараторы в начале и конце токена
                sep_start = token_str if token_str and token_str[0] in SEPARATORS else None
                sep_end = token_str if token_str and token_str[-1] in SEPARATORS and len(token_str) > 1 else None
                is_only_separator = all(c in SEPARATORS for c in token_str) if token_str else False

########################################################
                # 1) Первый токен в последовательности
                if not current_word:
                    if sep_start:
                        # Убираем сепаратор из начала и продолжаем генерацию
                        remaining_part = token_str[1:]
                        if remaining_part:
                            # Ищем токен по оставшейся части
                            for tid in cyrillic_token_ids:
                                if token_cache[tid]['str'] == remaining_part:
                                    new_beams.append((new_ids, [tid], [prob], total_prob, current_depth + 1))
                                    break
                        # Если токен состоит только из сепаратора - пропускаем (не сохраняем)
                    else:
                        # Начинаем новое слово
                        new_beams.append((new_ids, [token_id], [prob], total_prob, current_depth + 1))
                    continue
########################################################
                # 2) Не первый токен и начинается с сепаратора
                if sep_start and not is_only_separator:
                    # Сохраняем текущее слово
                    word_str = ''.join([token_cache[t]['str'] for t in current_word])
                    if is_single_cyrillic_word(word_str):
                        if word_str not in completed_words or current_prob > completed_words[word_str]:
                            completed_words[word_str] = current_prob
                            word_details[word_str] = {
                                'tokens': current_word.copy(),
                                'probs': current_word_probs.copy(),
                                'depth': current_depth,
                                'stop_reason': {sep_start},
                                'separator_probability': prob
                            }

                    # Прекращаем генерацию для этого луча
                    continue
########################################################
                # 3) Токен состоит только из сепаратора
                if is_only_separator:
                    # Сохраняем текущее слово
                    if current_word:
                        word_str = ''.join([token_cache[t]['str'] for t in current_word])
                        if is_single_cyrillic_word(word_str):
                            if word_str not in completed_words or current_prob > completed_words[word_str]:
                                completed_words[word_str] = current_prob
                                word_details[word_str] = {
                                    'tokens': current_word.copy(),
                                    'probs': current_word_probs.copy(),
                                    'depth': current_depth,
                                    'stop_reason': {token_str},
                                    'separator_probability': prob
                                }

                    # Прекращаем генерацию для этого луча
                    continue
########################################################
                # 4) Токен заканчивается сепаратором
                if sep_end:
                    # Добавляем буквенную часть к текущему слову
                    word_part = token_str[:-1]
                    if word_part:
                        # Ищем токен по буквенной части
                        for tid in cyrillic_token_ids:
                            if token_cache[tid]['str'] == word_part:
                                new_word = current_word + [tid]
                                new_word_probs = current_word_probs + [prob]
                                word_str = ''.join([token_cache[t]['str'] for t in new_word])
                                if is_single_cyrillic_word(word_str):
                                    if word_str not in completed_words or total_prob > completed_words[word_str]:
                                        completed_words[word_str] = total_prob
                                        word_details[word_str] = {
                                            'tokens': new_word.copy(),
                                            'probs': new_word_probs.copy(),
                                            'depth': current_depth + 1,
                                            'stop_reason': {sep_end},
                                            'separator_probability': prob
                                        }
                                break

                    # Прекращаем генерацию для этого луча
                    continue
########################################################
                # 5) Токен без сепараторов - продолжаем генерацию
                new_word = current_word + [token_id]
                new_word_probs = current_word_probs + [prob]
                new_beams.append((new_ids, new_word, new_word_probs, total_prob, current_depth + 1))

        # Сортируем и выбираем лучшие лучи
        new_beams.sort(key=lambda x: x[3], reverse=True)
        beams = new_beams[:beam_width]

    # Добавляем последние незавершённые слова
    for beam in beams:
        current_ids, current_word, current_word_probs, current_prob, current_depth = beam
        if current_word:
            word_str = ''.join([token_cache[t]['str'] for t in current_word])
            if is_single_cyrillic_word(word_str):
                if word_str not in completed_words or current_prob > completed_words[word_str]:
                    completed_words[word_str] = current_prob
                    word_details[word_str] = {
                        'tokens': current_word.copy(),
                        'probs': current_word_probs.copy(),
                        'depth': current_depth,
                        'stop_reason': "достигнута максимальная глубина",
                        'separator_probability': None
                    }

    results = []
    for word, prob in completed_words.items():
        details = word_details[word]
        results.append({
            'word': word,
            'probability': prob,
            'token_probabilities': details['probs'],
            'token_count': len(details['tokens']),
            'depth': details['depth'],
            'tokens': details['tokens'],
            'token_texts': [token_cache[t]['str'] for t in details['tokens']],
            'stop_reason': details['stop_reason'],
            'separator_probability': details['separator_probability']
        })

    results.sort(key=lambda x: x['probability'], reverse=True)
    return results

In [25]:
from datetime import datetime

def create_complete_probability_table3(sentences, tokenizer, model,
                                      max_depth=5, beam_width=50, min_prob=1e-4,
                                      output_path=None):
    """
    Создает таблицу с вероятностями полных слов и сохраняет результаты в CSV-файл.

    Для долгосрочного хранения результатов рекомендуется указать путь к папке на Google Диске.

    Позже можно будет объединить все CSV-файлы из этой папки в одну таблицу для анализа.
    """
    device = model.device
    cyrillic_token_ids, new_id_to_old_id, old_id_to_new_id, token_cache = initialize_token_structures(tokenizer)
    all_results = []
    for sentence_idx, sentence in enumerate(sentences, 1):
        print(f"Обработка предложения {sentence_idx}: {sentence}")
        completions = beam_search_completions3(
            sentence, tokenizer, model,
            max_depth=max_depth,
            beam_width=beam_width,
            min_prob=min_prob,
            cyrillic_token_ids=cyrillic_token_ids,
            new_id_to_old_id=new_id_to_old_id,
            token_cache=token_cache,
            device=device
        )
        if completions:
            for completion in completions:
                completion['sentence_id'] = sentence_idx
                completion['sentence_text'] = sentence
                completion.setdefault('separator_probability', None)
                all_results.append(completion)
            print(f"  Найдено {len(completions)} вариантов")
        else:
            print(f"  Варианты не найдены")
    if all_results:
        df = pd.DataFrame(all_results)
        columns = ['sentence_id', 'sentence_text', 'word', 'probability',
                  'token_probabilities', 'token_count', 'depth', 'tokens',
                  'token_texts', 'stop_reason', 'separator_probability']
        df = df[columns]
        if output_path is None:
            output_path = f'/content/drive/MyDrive/Predictions_saved_14_09/{sentences[:10]}.csv'
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Результаты сохранены в: {output_path}")
        print(f"Всего вариантов: {len(df)}")
        return df
    else:
        print("Нет результатов для сохранения.")
        return None

In [26]:
def combine_csv_results(folder_path, output_file="combined_results.csv"):
    """
    Объединяет все CSV-файлы с результатами из указанной папки.

    Параметры:
    folder_path - путь к папке с CSV-файлами
    output_file - имя результирующего файла
    """
    import glob
    import os

    # Находим все CSV-файлы в папке
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    if not csv_files:
        print("В указанной папке не найдено CSV-файлов")
        return None

    # Читаем и объединяем все файлы
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        df['source_file'] = os.path.basename(file)  # Добавляем информацию о источнике
        dataframes.append(df)

    # Объединяем все DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Сохраняем объединенный результат
    combined_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Объединенные результаты сохранены в: {output_file}")
    print(f"Общее количество записей: {len(combined_df)}")

    return combined_df

In [27]:
from google.colab import drive
import os

In [28]:
import time
import threading
from IPython.display import display, Javascript

class ColabKeeper:
    def __init__(self):
        self.keep_running = True

    def js_keep_alive(self):
        display(Javascript('''
        function ClickConnect(){
            console.log("Keeping Colab alive");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 55000)
        '''))

    def python_keep_alive(self):
        while self.keep_running:
            time.sleep(30)
            print(f"🔄 Activity: {time.ctime()}")

    def start(self):
        self.js_keep_alive()
        thread = threading.Thread(target=self.python_keep_alive, daemon=True)
        thread.start()

    def stop(self):
        self.keep_running = False

keeper = ColabKeeper()
keeper.start()

<IPython.core.display.Javascript object>

In [29]:
from datetime import datetime

In [ ]:
%%time

if __name__ == "__main__":
    # Пример использования
    test_sentences = [
        "В котел бросают куски <mask>",
        "А что получится теперь <mask>"

    ]
    # gdrive_folder_path = '/content/drive/MyDrive/результатами'
    # os.makedirs(gdrive_folder_path, exist_ok=True)
    probability_table = create_complete_probability_table3(
        sentences=test_sentences,
        tokenizer=tokenizer,
        model=model,
        max_depth=20,
        beam_width=200,
        min_prob=1e-6
    )


Reduced vocabulary size: 6529 / 128000
Обработка предложения 1: В котел бросают куски <mask>
🔄 Activity: Sun Sep 14 13:34:33 2025
🔄 Activity: Sun Sep 14 13:35:03 2025
🔄 Activity: Sun Sep 14 13:35:33 2025
🔄 Activity: Sun Sep 14 13:36:03 2025
🔄 Activity: Sun Sep 14 13:36:33 2025
🔄 Activity: Sun Sep 14 13:37:03 2025
🔄 Activity: Sun Sep 14 13:37:33 2025
  Найдено 486 вариантов
Обработка предложения 2: А что получится теперь <mask>
🔄 Activity: Sun Sep 14 13:38:03 2025
🔄 Activity: Sun Sep 14 13:38:33 2025
🔄 Activity: Sun Sep 14 13:39:03 2025
🔄 Activity: Sun Sep 14 13:39:33 2025
🔄 Activity: Sun Sep 14 13:40:03 2025
🔄 Activity: Sun Sep 14 13:40:33 2025
  Найдено 315 вариантов
Результаты сохранены в: /content/drive/MyDrive/Predictions_saved_14_09/['В котел бросают куски <mask>', 'А что получится теперь <mask>'].csv
Всего вариантов: 801
CPU times: user 6min 24s, sys: 711 ms, total: 6min 25s
Wall time: 6min 47s


In [30]:
left_contexts = ["В тот момент <mask>",
"Клиенты воровали из ресторана <mask>",
"Ему удалось вскрыть банку об острый край <mask>",
"Ей никак не суметь <mask>",
"Старуха была страшной -- <mask>",
"Выбирая вязаную шапочку, знайте, что лучше шапка цвета <mask>",
"У моего отца был счёт в швейцарском <mask>",
"Очень хочется заплести <mask>",
"В котёл бросают куски <mask>",
"На запись голоса <mask>",
"Ее сын Гриша умер <mask>",
"Можно будет <mask>",
"Музыканты играли на похоронах, разгружали <mask>",
"И не надо ставить это целью <mask>",
"Дрозды и сковрцы начали <mask>",
"Что может сделать самый сильный <mask>",
"Здесь потребуется <mask>",
"Какие главные лекарства должны <mask>",
"Применение микросхемы <mask>",
"Выходя замуж, ты надеялась обрести спокойствие, уютный <mask>",
"С нескрываемой <mask>",
"Однако здесь <mask>",
"Тому, кто <mask>",
"В качестве примера приводится <mask>",
"Он признаёт право каждого <mask>",
"Вспоминая <mask>",
"Он вскрыл пачку сухарей, <mask>",
"Но четыре года я не мог себя <mask>",
"На Ольгу Васильевну было написано <mask>",
"Я знал: их особенно <mask>",
"Очень тогда <mask>",
"Создать настоящие шедевры вам помогут <mask>",
"Наши власти позволяют себе <mask>",
"У нас в Волгограде многие придерживаются <mask>",
"Во избежание ожогов надо нанести на лицо небольшое <mask>",
"В вопросе послышался упрёк <mask>",
"За углом ― Морской музей, с бесчисленными моделями <mask>",
"Мне нравится сын коллеги, <mask>",
"Душа требовала <mask>",
"Наше правительство сделало <mask>",
"А промывать манную <mask>",
"Каждое утро на самый верх <mask>",
"Убедительно просим вас разборчиво заполнять <mask>",
"На болотах оставался ещё <mask>",
"Дорога ведет в глухой <mask>",
"На ведущей вниз <mask>",
"Когда она в самолёте <mask>",
"Возможности этих перемен будут обсуждаться в Париже <mask>",
"Там, недалеко от кухонной двери, сидел <mask>",
"В багажнике были лопата, <mask>",
"Врач прописал заживляющую <mask>",
"В бассейне <mask>",
"В резервациях <mask>",
"Мама брала меня с собой, и мы, сдав <mask>",
"Приблизительно в центре тайги <mask>",
"Не поручайту <mask>",
"В деревнях по-прежнему <mask>",
"В числе возможных кандидатов <mask>",
"Твоё тело расслабляется, и исчезает <mask>",
"Он был очень <mask>",
"Они не ели целый день, <mask>",
"В речи учёного прозвучало <mask>",
"Тема эта в то время была <mask>",
"Существует легенда, что <mask>",
"Он ловко поддел концом <mask>",
"У директора школы был тонкий <mask>",
"У Пашки <mask>",
"В конверте вместе с деньгами была <mask>",
"Стала стабильнее экономическая и политическая <mask>",
"В современном <mask>",
"Педагог предъявляет <mask>",
"Я сказал, что русский солдат <mask>",
"Старый шкаф <mask>",
"В сюжете этого фильма какие-то <mask>",
"Ирине досталась <mask>",
"На газовой плите стояла <mask>",
"Я люблю салат из картошки с зеленью, заправленный <mask>",
"Ненужный коврик из твёрдой <mask>",
"Что ты хочешь чтобы тебе <mask>",
"Журналист взял карандаш, <mask>",
"У мамы есть <mask>",
"Отвернув цветастое <mask>",
"У тебя впереди замечательный день, <mask>",
"Этот студент <mask>",
"Она почти не изменилась, только слегка <mask>",
"Перед ним снова была <mask>",
"Торговля продуктами питания является одной из самых <mask>",
"Когда родители <mask>",
"Товарищ генерал, <mask>",
"Считается, что коллекционирование <mask>",
"Телята быстро <mask>",
"Один футболист, который получил <mask>",
"Судя по огромному <mask>",
"Город, раскинувшийся вдоль <mask>",
"Под рукавом рубашки виднелся тонкий <mask>",
"На вторичном рынке жилья <mask>",
"Ваня раскрыл было <mask>",
"В мои обязанности входило утром включить <mask>",
"И на берегу озера тогда появляются <mask>",
"Наживка, на которую он ловил <mask>",
"Не обнаружив ничего в досье, сыщики решили <mask>",
"Когда мне хотелось <mask>",
"Взяв с собой фотоаппарат, вся <mask>",
"Собаку, виновницу случившегося, приказали <mask>",
"Мне было лень идти на стоянку и сметать <mask>",
"От смерти его спасла <mask>",
"По воскресеньям музыканты, исполнявшие <mask>",
"Он умел из любого <mask>",
"Если я еще увижу здесь хоть <mask>",
"Количество денег в обороте выросло благодаря <mask>",
"Он стал плохо <mask>",
"Работы выполняет <mask>",
"Зачем ему звонить, если откликается <mask>",
"Я слезал, щупал <mask>",
"Шею Лизы украшало ожерелье <mask>",
"Этот роман захватывает читателя с первой <mask>",
"Автор принадлежит к числу последних свидетелей <mask>",
"В темноте Иван задел острый <mask>",
"Приготовь себе диетические овощные блюда, <mask>",
"Олень бродил среди берёз, жевал <mask>",
"Причиной аварии был мобильный <mask>",
"После завершения <mask>",
"Чтобы придать объем <mask>",
"В лесу ветром <mask>",
"В каждом <mask>",
"Власть судов была такой <mask>",
"Володя каким-то образом <mask>",
"За два года накопилась <mask>",
"Если мы позволим этим людям <mask>",
"Думаю, большой <mask>",
"Елена сидела в кресле, молодая Мурка <mask>",
"От внимания наблюдателя не должна <mask>",
"Государством предлагается <mask>",
"Сделав мне знак помолчать, он приложил <mask>",
"Она успевала убраться, разморозить <mask>",
"Что используют для этой прически, <mask>",
"Она с досадой <mask>",
"Зоопарк ― это кусочек другого мира, находящийся в самом <mask>",
"Я сделала <mask>",
"За министром труда тянется целый <mask>",
"Мы установили камеру на новый <mask>",
"Ей хотелось выплеснуть чай на бежевый <mask>",
"На привале у озера <mask>",
"Покуда я нахожусь у власти, я буду предметом <mask>"]

In [ ]:
%%time

if __name__ == "__main__":

  for context in left_contexts:

    probability_table = create_complete_probability_table3(
          sentences=[context],
          tokenizer=tokenizer,
          model=model,
          max_depth=20,
          beam_width=200,
          min_prob=1e-6
    )


Reduced vocabulary size: 6529 / 128000
Обработка предложения 1: В тот момент <mask>
🔄 Activity: Sun Sep 14 14:06:47 2025
🔄 Activity: Sun Sep 14 14:07:17 2025
🔄 Activity: Sun Sep 14 14:07:47 2025
🔄 Activity: Sun Sep 14 14:08:17 2025
🔄 Activity: Sun Sep 14 14:08:47 2025
🔄 Activity: Sun Sep 14 14:09:17 2025
  Найдено 288 вариантов
Результаты сохранены в: /content/drive/MyDrive/Predictions_saved_14_09/['В тот момент <mask>'].csv
Всего вариантов: 288
Reduced vocabulary size: 6529 / 128000
Обработка предложения 1: Клиенты воровали из ресторана <mask>
🔄 Activity: Sun Sep 14 14:09:47 2025
🔄 Activity: Sun Sep 14 14:10:17 2025
🔄 Activity: Sun Sep 14 14:10:47 2025
🔄 Activity: Sun Sep 14 14:11:17 2025
